# <font style="color:rgb(50,120,229)">Video Stabilization Using Point Feature Matching</font>

In this module, we will learn how to implement a simple Video Stabilizer using a technique called Point Feature Matching in OpenCV library. We will discuss the algorithm and share the code(in python) to design a simple stabilizer using this method in OpenCV.

## <font style="color:rgb(50,120,229)">Video Stabilization</font>

**Video stabilization** refers to a family of methods used to reduce the effect of camera motion on the final video. The motion of the camera would be a translation ( i.e. movement in the x, y, z-direction ) or rotation (yaw, pitch, roll).


Have a look at [this](https://www.youtube.com/watch?v=NMGNak2u3Rc) video and notice the Low-Frequency camera motion.

## <font style="color:rgb(50,120,229)">Applications of Video Stabilization</font>

The need for video stabilization spans many domains.

It is extremely important in consumer and professional **videography**. Therefore, many different mechanical, optical, and algorithmic solutions exist. Even in still image **photography**, stabilization can help take handheld pictures with long exposure times.

In **medical diagnostic** applications like endoscopy and colonoscopy, videos need to be stabilized to determine the exact location and width of the problem.

Similarly, in **military applications**, videos captured by aerial vehicles on a reconnaissance flight need to be stabilized for localization, navigation, target tracking, etc. The same applies to **robotic applications**.

## <font style="color:rgb(50,120,229)">Different Approaches to Video Stabilization</font>

Video Stabilization approaches include mechanical, optical and digital stabilization methods. These are discussed briefly below:

- **Mechanical Video Stabilization**: Mechanical image stabilization systems use the motion detected by special sensors like gyros and accelerometers to move the image sensor to compensate for the motion of the camera.

- **Optical Video Stabilization**: In this method, instead of moving the entire camera, stabilization is achieved by moving parts of the lens. This method employs a moveable lens assembly that variably adjusts the path length of light as it travels through the camera’s lens system.

- **Digital Video Stabilization**:  This method does not require special sensors for estimating camera motion. There are three main steps — 1) motion estimation 2) motion smoothing, and 3) image composition. The transformation parameters between two consecutive frames are derived in the first stage. The second stage filters out unwanted motion and in the last stage the stabilized video is reconstructed.

We will learn a fast and robust implementation of a digital video stabilization algorithm in this post. It is based on a two-dimensional motion model where we apply a **Euclidean (a.k.a Similarity) transformation** incorporating translation, rotation, and scaling.

![](https://www.dropbox.com/s/8fc343kafgmljx3/motion_models.png?dl=1)

As you can see in the image above, in a Euclidean motion model, a square in an image can transform to any other square with a different location, size or rotation. It is more restrictive than affine and homography transforms but is adequate for motion stabilization because the camera movement between successive frames of a video is usually small.

## <font style="color:rgb(50,120,229)">Video Stabilization Using Point Feature Matching</font>

This method involves tracking a few feature points between two consecutive frames. The tracked features allow us to estimate the motion between frames and compensate for it.

The flowchart below shows the basic steps.

&nbsp;

<center><a href="https://www.learnopencv.com/wp-content/uploads/2019/01/AEAM-3.png"><img src="https://www.learnopencv.com/wp-content/uploads/2019/01/AEAM-3.png" /></a></center>

Let’s go over the steps.

### <font style="color:rgb(50,120,229)">Step 1 : Set Input and Output Videos</font>

First, let’s complete the setup for reading the input video and writing the output video. The comments in the code explain every line.

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from dataPath import DATA_PATH
%matplotlib inline

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0,6.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [3]:
# Read input video
cap = cv2.VideoCapture(DATA_PATH+'videos/video.mp4')

In [4]:
# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [5]:
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [6]:
# Get frames per second (fps)
fps = cap.get(cv2.CAP_PROP_FPS)

In [7]:
# Set up output video
out = cv2.VideoWriter('video_out.avi', cv2.VideoWriter_fourcc('M','J','P','G'), fps, (w*2, h))

In [8]:
# The larger the more stable the video, but less reactive to sudden panning
SMOOTHING_RADIUS=50 

### <font style="color:rgb(50,120,229)">Step 2: Read the first frame and convert it to grayscale</font>

For video stabilization, we need to capture two frames of a video, estimate motion between the frames, and finally correct the motion.

In [9]:
# Read first frame
_, prev = cap.read()

In [10]:
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

### <font style="color:rgb(50,120,229)">Step 3: Find motion between frames</font>

This is the most crucial part of the algorithm. We will iterate over all the frames, and find the motion between the current frame and the previous frame. It is not necessary to know the motion of each and every pixel. The Euclidean motion model requires that we know the motion of only 2 points in the two frames. However, in practice, it is a good idea to find the motion of 50-100 points, and then use them to robustly estimate the motion model.

### <font style="color:rgb(8,133,37)">3.1 Good Features to Track</font>

The question now is what points should we choose for tracking. Keep in mind that tracking algorithms use a small patch around a point to track it. Such tracking algorithms suffer from the **aperture problem**.

So, smooth regions are bad for tracking and textured regions with lots of corners are good. Fortunately, OpenCV has a fast feature detector that detects features that are ideal for tracking. It is called **`goodFeaturesToTrack`** (no kidding!).


#### <font style="color:rgb(8,133,37)">Function Syntax</font>

```python
corners	=	cv2.goodFeaturesToTrack(	image, maxCorners, qualityLevel, minDistance, mask, blockSize, gradientSize[, corners[, useHarrisDetector[, k]]]	)
```

Where,

- **`image`** - Input 8-bit or floating-point 32-bit, single-channel image.
- **`corners`** - Output vector of detected corners.
- **`maxCorners`** - Maximum number of corners to return. If there are more corners than are found, the strongest of them is returned. `maxCorners <= 0` implies that no limit on the maximum is set and all detected corners are returned.
- **`qualityLevel`** - Parameter characterizing the minimal accepted quality of image corners. The parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue or the Harris function response. The corners with the quality measure less than the product are rejected. For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure less than 15 are rejected.
- **`minDistance`** - Minimum possible Euclidean distance between the returned corners.
- **`mask`** - Optional region of interest. If the image is not empty (it needs to have the type CV_8UC1 and the same size as image ), it specifies the region in which the corners are detected.
- **`blockSize`** - Size of an average block for computing a derivative covariation matrix over each pixel neighborhood.
- **`useHarrisDetector`** - Parameter indicating whether to use a Harris detector or `cornerMinEigenVal`.
- **`k`** - Free parameter of the Harris detector.

### <font style="color:rgb(8,133,37)">3.2 Lucas-Kanade Optical Flow</font>

Once we have found good features in the previous frame, we can track them in the next frame using an algorithm called **Lucas-Kanade Optical Flow** named after the inventors of the algorithm.

It is implemented using the function **`calcOpticalFlowPyrLK`** in OpenCV. In the name `calcOpticalFlowPyrLK`, **LK** stands for Lucas-Kanade, and **Pyr** stands for the pyramid. An image pyramid in computer vision is used to process an image at different scales (resolutions).

#### <font style="color:rgb(8,133,37)">Function Syntax</font>

```python
nextPts, status, err	=	cv2.calcOpticalFlowPyrLK(	prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]	)
```

Where,

- **`prevImg`** - first 8-bit input image or pyramid constructed by buildOpticalFlowPyramid.
- **`nextImg`** - second input image or pyramid of the same size and the same type as prevImg.
- **`prevPts`** - vector of 2D points for which the flow needs to be found; point coordinates must be single-precision floating-point numbers.
- **`nextPts`** - output vector of 2D points (with single-precision floating-point coordinates) containing the calculated new positions of input features in the second image; when OPTFLOW_USE_INITIAL_FLOW flag is passed, the vector must have the same size as in the input.
- **`status`** - output status vector (of unsigned chars); each element of the vector is set to 1 if the flow for the corresponding features has been found, otherwise, it is set to 0.
- **`err`** - output vector of errors; each element of the vector is set to an error for the corresponding feature, type of the error measure can be set in flags parameter; if the flow wasn't found then the error is not defined (use the status parameter to find such cases).
- **`winSize`** - size of the search window at each pyramid level.
- **`maxLevel`** -0-based maximal pyramid level number; if set to 0, pyramids are not used (single level), if set to 1, two levels are used, and so on; if pyramids are passed to input then algorithm will use as many levels as pyramids have but no more than maxLevel.
- **`criteria`** - parameter, specifying the termination criteria of the iterative search algorithm (after the specified maximum number of iterations criteria.maxCount or when the search window moves by less than criteria.epsilon.
- **`flags`** - operation flags:
  - `OPTFLOW_USE_INITIAL_FLOW` uses initial estimations, stored in nextPts; if the flag is not set, then prevPts is copied to nextPts and is considered the initial estimate.
  - `OPTFLOW_LK_GET_MIN_EIGENVALS` use minimum eigen values as an error measure (see minEigThreshold description); if the flag is not set, then L1 distance between patches around the original and a moved point, divided by number of pixels in a window, is used as a error measure.
- **`minEigThreshold`** - the algorithm calculates the minimum eigen value of a 2x2 normal matrix of optical flow equations (this matrix is called a spatial gradient matrix), divided by number of pixels in a window; if this value is less than minEigThreshold, then a corresponding feature is filtered out and its flow is not processed, so it allows to remove bad points and get a performance boost.

**`calcOpticalFlowPyrLK`** may not be able to calculate the motion of all the points because of a variety of reasons. For example, the feature point in the current frame could get occluded by another object in the next frame. Fortunately, as you will see in the code below, the **status** flag in **`calcOpticalFlowPyrLK`** can be used to filter out these values.

### <font style="color:rgb(8,133,37)">3.3 Estimate Motion</font>

To recap, in step 3.1, we found good features to track in the previous frame. In step 3.2, we used optical flow to track the features. In other words, we found the location of the features in the current frame, and we already knew the location of the features in the previous frame. So we can use these two sets of points to find the rigid (Euclidean) transformation that maps the previous frame to the current frame. This is done using the function **`estimateRigidTransform`**.

Once we have estimated the motion, we can decompose it into x and y translation and rotation (angle). We store these values in an array so we can change them smoothly.

The code below goes over steps 3.1 to 3.3. Make sure to read the comments in the code to follow along.

In [11]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32)

In [12]:
for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)
    
  # Read next frame
  success, curr = cap.read() 
  if not success: 
    break
 
  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 
 
  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
 
  # Sanity check
  assert prev_pts.shape == curr_pts.shape 
 
  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]
 
  #Find transformation matrix
  m = cv2.estimateAffinePartial2D(prev_pts, curr_pts)
  # Extract traslation
  dx = m[0][0,2]
  dy = m[0][1,2]
 
  # Extract rotation angle
  da = np.arctan2(m[0][1,0], m[0][0,0])
    
  # Store transformation
  transforms[i] = [dx,dy,da]
    
  # Move to next frame
  prev_gray = curr_gray
 
  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/1362 -  Tracked points : 106
Frame: 1/1362 -  Tracked points : 108
Frame: 2/1362 -  Tracked points : 104
Frame: 3/1362 -  Tracked points : 106
Frame: 4/1362 -  Tracked points : 107
Frame: 5/1362 -  Tracked points : 107
Frame: 6/1362 -  Tracked points : 110
Frame: 7/1362 -  Tracked points : 110
Frame: 8/1362 -  Tracked points : 102
Frame: 9/1362 -  Tracked points : 109
Frame: 10/1362 -  Tracked points : 104
Frame: 11/1362 -  Tracked points : 107
Frame: 12/1362 -  Tracked points : 109
Frame: 13/1362 -  Tracked points : 108
Frame: 14/1362 -  Tracked points : 104
Frame: 15/1362 -  Tracked points : 109
Frame: 16/1362 -  Tracked points : 103
Frame: 17/1362 -  Tracked points : 106
Frame: 18/1362 -  Tracked points : 104
Frame: 19/1362 -  Tracked points : 110
Frame: 20/1362 -  Tracked points : 102
Frame: 21/1362 -  Tracked points : 104
Frame: 22/1362 -  Tracked points : 103
Frame: 23/1362 -  Tracked points : 103
Frame: 24/1362 -  Tracked points : 102
Frame: 25/1362 -  Tracked points : 

Frame: 222/1362 -  Tracked points : 103
Frame: 223/1362 -  Tracked points : 101
Frame: 224/1362 -  Tracked points : 101
Frame: 225/1362 -  Tracked points : 108
Frame: 226/1362 -  Tracked points : 102
Frame: 227/1362 -  Tracked points : 104
Frame: 228/1362 -  Tracked points : 102
Frame: 229/1362 -  Tracked points : 101
Frame: 230/1362 -  Tracked points : 105
Frame: 231/1362 -  Tracked points : 102
Frame: 232/1362 -  Tracked points : 100
Frame: 233/1362 -  Tracked points : 99
Frame: 234/1362 -  Tracked points : 95
Frame: 235/1362 -  Tracked points : 103
Frame: 236/1362 -  Tracked points : 99
Frame: 237/1362 -  Tracked points : 108
Frame: 238/1362 -  Tracked points : 111
Frame: 239/1362 -  Tracked points : 109
Frame: 240/1362 -  Tracked points : 104
Frame: 241/1362 -  Tracked points : 102
Frame: 242/1362 -  Tracked points : 101
Frame: 243/1362 -  Tracked points : 107
Frame: 244/1362 -  Tracked points : 104
Frame: 245/1362 -  Tracked points : 106
Frame: 246/1362 -  Tracked points : 109
Fra

Frame: 431/1362 -  Tracked points : 109
Frame: 432/1362 -  Tracked points : 108
Frame: 433/1362 -  Tracked points : 112
Frame: 434/1362 -  Tracked points : 107
Frame: 435/1362 -  Tracked points : 117
Frame: 436/1362 -  Tracked points : 112
Frame: 437/1362 -  Tracked points : 114
Frame: 438/1362 -  Tracked points : 111
Frame: 439/1362 -  Tracked points : 114
Frame: 440/1362 -  Tracked points : 115
Frame: 441/1362 -  Tracked points : 114
Frame: 442/1362 -  Tracked points : 114
Frame: 443/1362 -  Tracked points : 113
Frame: 444/1362 -  Tracked points : 110
Frame: 445/1362 -  Tracked points : 115
Frame: 446/1362 -  Tracked points : 121
Frame: 447/1362 -  Tracked points : 121
Frame: 448/1362 -  Tracked points : 117
Frame: 449/1362 -  Tracked points : 118
Frame: 450/1362 -  Tracked points : 116
Frame: 451/1362 -  Tracked points : 120
Frame: 452/1362 -  Tracked points : 117
Frame: 453/1362 -  Tracked points : 111
Frame: 454/1362 -  Tracked points : 117
Frame: 455/1362 -  Tracked points : 115


Frame: 641/1362 -  Tracked points : 107
Frame: 642/1362 -  Tracked points : 110
Frame: 643/1362 -  Tracked points : 116
Frame: 644/1362 -  Tracked points : 114
Frame: 645/1362 -  Tracked points : 116
Frame: 646/1362 -  Tracked points : 113
Frame: 647/1362 -  Tracked points : 111
Frame: 648/1362 -  Tracked points : 114
Frame: 649/1362 -  Tracked points : 113
Frame: 650/1362 -  Tracked points : 110
Frame: 651/1362 -  Tracked points : 115
Frame: 652/1362 -  Tracked points : 115
Frame: 653/1362 -  Tracked points : 113
Frame: 654/1362 -  Tracked points : 113
Frame: 655/1362 -  Tracked points : 108
Frame: 656/1362 -  Tracked points : 109
Frame: 657/1362 -  Tracked points : 114
Frame: 658/1362 -  Tracked points : 115
Frame: 659/1362 -  Tracked points : 113
Frame: 660/1362 -  Tracked points : 112
Frame: 661/1362 -  Tracked points : 115
Frame: 662/1362 -  Tracked points : 112
Frame: 663/1362 -  Tracked points : 117
Frame: 664/1362 -  Tracked points : 113
Frame: 665/1362 -  Tracked points : 113


Frame: 857/1362 -  Tracked points : 116
Frame: 858/1362 -  Tracked points : 116
Frame: 859/1362 -  Tracked points : 119
Frame: 860/1362 -  Tracked points : 113
Frame: 861/1362 -  Tracked points : 114
Frame: 862/1362 -  Tracked points : 118
Frame: 863/1362 -  Tracked points : 124
Frame: 864/1362 -  Tracked points : 114
Frame: 865/1362 -  Tracked points : 121
Frame: 866/1362 -  Tracked points : 117
Frame: 867/1362 -  Tracked points : 123
Frame: 868/1362 -  Tracked points : 119
Frame: 869/1362 -  Tracked points : 114
Frame: 870/1362 -  Tracked points : 117
Frame: 871/1362 -  Tracked points : 116
Frame: 872/1362 -  Tracked points : 113
Frame: 873/1362 -  Tracked points : 120
Frame: 874/1362 -  Tracked points : 116
Frame: 875/1362 -  Tracked points : 113
Frame: 876/1362 -  Tracked points : 116
Frame: 877/1362 -  Tracked points : 113
Frame: 878/1362 -  Tracked points : 110
Frame: 879/1362 -  Tracked points : 114
Frame: 880/1362 -  Tracked points : 115
Frame: 881/1362 -  Tracked points : 115


Frame: 1072/1362 -  Tracked points : 109
Frame: 1073/1362 -  Tracked points : 107
Frame: 1074/1362 -  Tracked points : 104
Frame: 1075/1362 -  Tracked points : 114
Frame: 1076/1362 -  Tracked points : 112
Frame: 1077/1362 -  Tracked points : 113
Frame: 1078/1362 -  Tracked points : 113
Frame: 1079/1362 -  Tracked points : 113
Frame: 1080/1362 -  Tracked points : 108
Frame: 1081/1362 -  Tracked points : 112
Frame: 1082/1362 -  Tracked points : 114
Frame: 1083/1362 -  Tracked points : 115
Frame: 1084/1362 -  Tracked points : 113
Frame: 1085/1362 -  Tracked points : 111
Frame: 1086/1362 -  Tracked points : 110
Frame: 1087/1362 -  Tracked points : 115
Frame: 1088/1362 -  Tracked points : 110
Frame: 1089/1362 -  Tracked points : 112
Frame: 1090/1362 -  Tracked points : 108
Frame: 1091/1362 -  Tracked points : 109
Frame: 1092/1362 -  Tracked points : 108
Frame: 1093/1362 -  Tracked points : 111
Frame: 1094/1362 -  Tracked points : 110
Frame: 1095/1362 -  Tracked points : 115
Frame: 1096/1362

Frame: 1274/1362 -  Tracked points : 121
Frame: 1275/1362 -  Tracked points : 114
Frame: 1276/1362 -  Tracked points : 112
Frame: 1277/1362 -  Tracked points : 115
Frame: 1278/1362 -  Tracked points : 117
Frame: 1279/1362 -  Tracked points : 116
Frame: 1280/1362 -  Tracked points : 116
Frame: 1281/1362 -  Tracked points : 112
Frame: 1282/1362 -  Tracked points : 111
Frame: 1283/1362 -  Tracked points : 122
Frame: 1284/1362 -  Tracked points : 114
Frame: 1285/1362 -  Tracked points : 116
Frame: 1286/1362 -  Tracked points : 111
Frame: 1287/1362 -  Tracked points : 118
Frame: 1288/1362 -  Tracked points : 118
Frame: 1289/1362 -  Tracked points : 116
Frame: 1290/1362 -  Tracked points : 119
Frame: 1291/1362 -  Tracked points : 114
Frame: 1292/1362 -  Tracked points : 112
Frame: 1293/1362 -  Tracked points : 118
Frame: 1294/1362 -  Tracked points : 124
Frame: 1295/1362 -  Tracked points : 121
Frame: 1296/1362 -  Tracked points : 114
Frame: 1297/1362 -  Tracked points : 116
Frame: 1298/1362

### <font style="color:rgb(50,120,229)">Step 4: Calculate smooth motion between frames</font>

In the previous step, we estimated the motion between the frames and stored them in an array. We now need to find the trajectory of motion by cumulatively adding the differential motion estimated in the previous step.

### <font style="color:rgb(8,133,37)">Step 4.1 : Calculate trajectory</font>

In this step, we will add up the motion between the frames to calculate the **trajectory**. Our ultimate goal is to smooth out this trajectory.

In Python, it is easily achieved using **`cumsum`** (cumulative sum) in numpy.

In [4]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)

### <font style="color:rgb(8,133,37)">Step 4.2 : Calculate smooth trajectory</font>

In the previous step, we calculated the trajectory of motion. So we have three curves that show how the motion (x, y, and angle) changes over time.

In this step, we will show how to smooth these three curves.

The easiest way to smooth any curve is to use a **moving average filter**. As the name suggests, a moving average filter replaces the value of a function at the point by the average of its neighbors defined by a window. Let’s look at an example.

Let’s say we have stored a curve in an array c, so the points on the curve are $c[0] … c[n-1]$. Let f be the smooth curve we obtain by filtering c with a moving average filter of width 5.

The $k^{th}$ element of this curve is calculated using

$$
  \begin{align*} f[k] = \frac{c[k-2] + c[k-1] + c[k] + c[k+1] + c[k+2]}{5} \end{align*}
$$

As you can see, the values of the smooth curve are the values of the noisy curve averaged over a small window. The figure below shows an example of the noisy curve on the left, smoothed using a box filter of size 5 on the right.

![](https://www.learnopencv.com/wp-content/uploads/2019/01/box-filtering.png)

In the Python implementation, we define a moving average filter that takes in any curve ( i.e. a 1-D of numbers) as an input and returns the smoothed version of the curve.

In [14]:
def movingAverage(curve, radius): 
  window_size = 2 * radius + 1
  # Define the filter 
  f = np.ones(window_size)/window_size 
  # Add padding to the boundaries 
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
  # Apply convolution 
  curve_smoothed = np.convolve(curve_pad, f, mode='same') 
  # Remove padding 
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed

We also define a function that takes in the trajectory and performs smoothing on the three components.

In [15]:
def smooth(trajectory): 
  smoothed_trajectory = np.copy(trajectory) 
  # Filter the x, y and angle curves
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)
 
  return smoothed_trajectory

And, here is the final usage.



In [16]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)

### <font style="color:rgb(8,133,37)">Step 4.3 : Calculate smooth transforms</font>

So far we have obtained a smooth trajectory. In this step, we will use the smooth trajectory to obtain smooth transforms that can be applied to frames of the videos to stabilize it.

This is done by finding the difference between the smooth trajectory and the original trajectory and adding this difference back to the original transforms.

In [17]:
# Create variable to store smoothed trajectory
smoothed_trajectory = smooth(trajectory) 

In [18]:
# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory
  
# Calculate newer transformation array
transforms_smooth = transforms + difference

### <font style="color:rgb(50,120,229)">Step 5: Apply smoothed camera motion to frames</font>

We are almost done. All we need to do now is to loop over the frames and apply the transforms we just calculated.

If we have a motion specified as $(x, y, \theta)$, the corresponding transformation matrix is given by

$$\begin{align*} T = \begin{bmatrix} \cos \theta & -\sin \theta & x \\ \sin \theta & \cos \theta & y \\ \end{bmatrix} \end{align*}$$

Read the comments in the code to follow along.

### <font style="color:rgb(8,133,37)">Step 5.1 : Fix border artifacts</font>

When we stabilize a video, we may see some black boundary artifacts. This is expected because to stabilize the video, a frame may have to shrink in size.

We can mitigate the problem by scaling the video about its center by a small amount (e.g. 4%).

The function **`fixBorder`** below shows the implementation. We use **`getRotationMatrix2D`** because it scales and rotates the image without moving the center of the image. All we need to do is call this function with 0 rotation and scale 1.04 ( i.e. 4% upscale).

In [19]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame

In [20]:
# Reset stream to first frame 
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [21]:
# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  success, frame = cap.read() 
  if not success:
    break
 
  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]
 
  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy
 
  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))
 
  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 
 
  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])
 
  # If the image is too big, resize it.
  if(frame_out.shape[1] > 1920): 
    frame_out = cv2.resize(frame_out, (w,h))
  #cv2.imshow("Frame",frame_out)
  #cv2.waitKey(0)

  out.write(frame_out)

In [22]:
cv2.destroyAllWindows()
out.release()

## <font style="color:rgb(50,120,229)">Result</font>

&nbsp;
<center><video controls="controls">
<source src="https://www.dropbox.com/s/lamfisu8c3yvr4v/video_out.avi?dl=1" type="video/avi" />
</video></center>


The result of the stabilization code we have shared is shown above. Our objective was to reduce the motion significantly, but not to eliminate it completely.

We leave it to the reader to think of a modification of the code that will eliminate motion between frames completely. What could be the side effects if you try to eliminate all camera motion?

The current method only works for a fixed length video and not with a real-time feed.  We have to modify this method heavily to attain real-time video output which is out of the scope for this post but it is achievable.

## <font style="color:rgb(50,120,229)">Pros and Cons</font>
### <font style="color:rgb(8,133,37)">Pros</font>

1. This method provides good stability against low-frequency motion (slower vibrations).
2. This method has low memory consumption thereby ideal for Embedded devices(like Raspberry Pi).
3. This method is good against zooming(scaling) jitter in the video.

### <font style="color:rgb(8,133,37)">Cons</font>

1. This method performs poorly against high-frequency perturbations.
2. If there is a heavy motion blur, feature tracking will fail and the results would not be optimal.
3. This method is also not good with Rolling Shutter distortion.